In [3]:
 !pip install sentence-transformers transformers datasets scikit-learn

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('/content/McDonaldsMenuNutritionV2.csv')  # Adjust the path to your dataset

# Check the dataset structure
print(df.head())

# Combine the item name and nutrition information to create a context
df['context'] = df['Item'] + ": " + df['Calories'].astype(str) + " calories, " + \
                df['Total Fat (g)'].astype(str) + "g total fat, " + \
                df['Protein (g)'].astype(str) + "g protein."

# Load the Sentence Transformer model for embeddings
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the combined context of the items
embeddings = retrieval_model.encode(df['context'].tolist())

# Load the T5 model and tokenizer for answer generation
generator_model = T5ForConditionalGeneration.from_pretrained('t5-base')
generator_tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Retrieve the most relevant document
def retrieve_document(query):
    query_embedding = retrieval_model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_index = similarities[0].argmax()  # Get the index of the most similar document
    return df['context'].iloc[top_index]  # Return the full context of the best match

# Generate an answer using the T5 model
def generate_answer(query, retrieved_doc):
    input_text = f"question: {query} context: {retrieved_doc}"
    input_ids = generator_tokenizer.encode(input_text, return_tensors='pt', truncation=True)

    with torch.no_grad():  # Save memory during inference
        output_ids = generator_model.generate(input_ids, max_length=50)

    return generator_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Main function to answer a query
def answer_query(query):
    retrieved_doc = retrieve_document(query)
    answer = generate_answer(query, retrieved_doc)
    return answer

# Interactive loop for user input
def interactive_rag_system():
    print("Welcome to the Interactive RAG System!")
    while True:
        user_input = input("Enter your query (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        else:
            try:
                answer = answer_query(user_input)
                print(f"Answer: {answer}")
            except Exception as e:
                print(f"An error occurred: {e}")

# Run the Interactive RAG System
if __name__ == "__main__":
    interactive_rag_system()


                            Item  Calories  Calories from Fat  Total Fat (g)  \
0                      Hamburger       250               80.0            9.0   
1                   Cheeseburger       300              110.0           12.0   
2            Double Cheeseburger       440              210.0           23.0   
3                       McDouble       390              170.0           19.0   
4   Quarter Pounder® with Cheese       510              230.0           26.0   

  Saturated Fat (g)  Trans Fat (g)  Cholesterol (mg)  Sodium (mg)  Carbs (g)  \
0               3.5            0.5                25          520         31   
1                 6            0.5                40          750         33   
2                11            1.5                80         1150         34   
3                 8            1.0                65          920         33   
4                12            1.5                90         1190         40   

   Fiber (g)  Sugars (g)  Protein (g) 

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Welcome to the Interactive RAG System!
Enter your query (or type 'exit' to quit): How much protein does a Quarter Pounder with Cheese have?


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Answer: 29g
Enter your query (or type 'exit' to quit): Tell me about the nutrition facts for a McDouble.
Answer: 390 calories, 19.0g total fat, 22g protein
Enter your query (or type 'exit' to quit): exit
